In [1]:
#import libraries (some of them may not be used)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
import keras
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
from keras.activations import sigmoid
from keras import layers
from keras import initializers
from keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Dropout, BatchNormalization,Embedding, LSTM, Reshape, Bidirectional,Conv1D, MaxPooling1D, AveragePooling1D, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint
import wave
import os.path
from pathlib import Path
from keras import regularizers
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.models import Sequential, Model, model_from_json, load_model
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from tensorflow.keras.utils import to_categorical, custom_object_scope
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm, tqdm_pandas
import scipy
import librosa
import librosa.display
import json
from matplotlib.pyplot import specgram
import seaborn as sns
import glob
import sys
import warnings
# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
#here we connect to google drive to access our datasets

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Feature Extraction (LPC and MFCC)

In [3]:
a1=[]
a2=[]
a3=[]

In [4]:
b1=[]
b2=[]
b3=[]

In [5]:
# extracting LPC features from the train data and put them in a list whit their filenames

for file in glob.glob("/content/drive/MyDrive/train_02/*.wav"):
    y, sr = librosa.load(file, sr=None)
    n_fft = 2048  # Frame size
    hop_length = 512  # Hop length for 75% overlap
    frames = librosa.util.frame(y, frame_length=n_fft, hop_length=hop_length).T
    lpc_features = []
    for frame in frames:
        lpc_coeffs = librosa.lpc(frame, order=20)
        lpc_features.append(lpc_coeffs)
    lpc_features = np.array(lpc_features)
    filename = Path(file).stem
    identifier = filename.split("_")[0]
    a1.append([identifier, lpc_features])


In [6]:
#extracting LPC features from the validation data and put them in a list whit their filenames

for file in glob.glob("/content/drive/MyDrive/valid_02/*.wav"):
    y, sr = librosa.load(file, sr=None)
    n_fft = 2048  # Frame size
    hop_length = 512  # Hop length for 75% overlap
    frames = librosa.util.frame(y, frame_length=n_fft, hop_length=hop_length).T
    lpc_features = []
    for frame in frames:
        lpc_coeffs = librosa.lpc(frame, order=20)
        lpc_features.append(lpc_coeffs)
    lpc_features = np.array(lpc_features)
    filename = Path(file).stem
    identifier = filename.split("_")[0]
    a2.append([identifier, lpc_features])

In [7]:
#extracting LPC features from the test data and put them in a list whit their filenames

for file in glob.glob("/content/drive/MyDrive/test_02/*.wav"):
    y, sr = librosa.load(file, sr=None)
    n_fft = 2048  # Frame size
    hop_length = 512  # Hop length for 75% overlap
    frames = librosa.util.frame(y, frame_length=n_fft, hop_length=hop_length).T
    lpc_features = []
    for frame in frames:
        lpc_coeffs = librosa.lpc(frame, order=20)
        lpc_features.append(lpc_coeffs)
    lpc_features = np.array(lpc_features)
    filename = Path(file).stem
    identifier = filename.split("_")[0]
    a3.append([identifier, lpc_features])

In [8]:
#extracting MFCC features from the train data and put them in a list whit their filenames

for file in glob.glob("/content/drive/MyDrive/train_02/*.wav"):

        y1, sr1 = librosa.load(file, sr = None, mono=True)
        mfccs1= librosa.feature.mfcc(y=y1, sr=sr1, n_mfcc=13)
        filename1 = Path(file).stem
        x1 = filename1.split("_")
        b1.append([x1[0],mfccs1])

In [9]:
#extracting MFCC features from the validation data and put them in a list whit their filenames

for file in glob.glob("/content/drive/MyDrive/valid_02/*.wav"):

        y2, sr2 = librosa.load(file, sr = None, mono=True)
        mfccs2= librosa.feature.mfcc(y=y2, sr=sr2,n_mfcc=13)
        filename2 = Path(file).stem
        x2 = filename2.split("_")
        b2.append([x2[0],mfccs2])

In [10]:
#extracting MFCC features from the test data and put them in a list whit their filenames

for file in glob.glob("/content/drive/MyDrive/test_02/*.wav"):

        y3, sr3 = librosa.load(file, sr = None, mono=True)
        mfccs3= librosa.feature.mfcc(y=y3, sr=sr3, n_mfcc=13)
        filename3 = Path(file).stem
        x3 = filename3.split("_")
        b3.append([x3[0],mfccs3])

In [11]:
len(a1)

460

Because the number of frames for each sample is differen, here we get the (Average + STD) of the frame numbers for each LPC-based and MFCC-based Dataset(train + valid + test).

In [12]:
s1 = []
s2 = []
s3 = []

In [13]:
f1 = []
f2 = []
f3 = []

In [14]:
for i in range (460):
  f1.append(len((b1[i][1]).T))
for i in range (105):
  f2.append(len((b2[i][1]).T))
for i in range (142):
  f3.append(len((b3[i][1]).T))

In [15]:
for i in range (460):
  s1.append(len((a1[i][1])))
for i in range (105):
  s2.append(len((a2[i][1])))
for i in range (142):
  s3.append(len((a3[i][1])))

In [16]:
s4 = s1 + s2 + s3

In [17]:
f4 = f1 + f2 + f3

In [18]:
F4 = np.array(f4)

In [19]:
S4 = np.array(s4)

In [20]:
ave = np.mean(F4)
s = np.std(F4)
print(min(F4))
print(ave)
print(s)
print(ave+s)

358
1370.025459688826
675.1442064427769
2045.1696661316028


In [21]:
ave = np.mean(S4)
s = np.std(S4)
print(min(S4))
print(ave)
print(s)
print(ave+s)

354
1366.025459688826
675.1442064427769
2041.1696661316028


Making X_train, X_test, X_valid and Y_train, Y_test, Y_valid for LPC and MFCC based datasets

In [22]:
c1=[]
c2=[]
c3=[]

In [23]:
d1=[]
d2=[]
d3=[]

In [24]:
for i in range(460):
        c1.append([a1[i][0], ((a1[i][1]))[0:2040]])
for i in range(105):
        c2.append([a2[i][0], ((a2[i][1]))[0:2040]])
for i in range(142):
        c3.append([a3[i][0], ((a3[i][1]))[0:2040]])

In [25]:
for i in range(460):
        d1.append([b1[i][0], ((b1[i][1]).T)[0:2045]])
for i in range(105):
        d2.append([b2[i][0], ((b2[i][1]).T)[0:2045]])
for i in range(142):
        d3.append([b3[i][0], ((b3[i][1]).T)[0:2045]])

In [26]:
data_train_LPC =[]
data_valid_LPC =[]
data_test_LPC =[]

In [27]:
data_train_MFCC =[]
data_valid_MFCC =[]
data_test_MFCC =[]

In [28]:
#we get our labels for each sample from this file

df = pd.read_csv('/content/drive/MyDrive/AVFAD_01_00_00.csv', encoding='ISO-8859-1')

In [29]:
for i in range(460):
    for j in range(709):
        if c1[i][0] == df.iloc[j,0]:
            data_train_LPC.append([c1[i][0], c1[i][1], df.iloc[j,15]])

for i in range(105):
    for j in range(709):
        if c2[i][0] == df.iloc[j,0]:
            data_valid_LPC.append([c2[i][0], c2[i][1], df.iloc[j,15]])

for i in range(142):
    for j in range(709):
        if c3[i][0] == df.iloc[j,0]:
            data_test_LPC.append([c3[i][0], c3[i][1], df.iloc[j,15]])

In [30]:
for i in range(460):
    for j in range(709):
        if d1[i][0] == df.iloc[j,0]:
            data_train_MFCC.append([d1[i][0], d1[i][1], df.iloc[j,15]])

for i in range(105):
    for j in range(709):
        if d2[i][0] == df.iloc[j,0]:
            data_valid_MFCC.append([d2[i][0], d2[i][1], df.iloc[j,15]])

for i in range(142):
    for j in range(709):
        if d3[i][0] == df.iloc[j,0]:
            data_test_MFCC.append([d3[i][0], d3[i][1], df.iloc[j,15]])

In [31]:
data_train_LPC.sort(key = lambda x: x[0])
data_valid_LPC.sort(key = lambda x: x[0])
data_test_LPC.sort(key = lambda x: x[0])

In [32]:
data_train_MFCC.sort(key = lambda x: x[0])
data_valid_MFCC.sort(key = lambda x: x[0])
data_test_MFCC.sort(key = lambda x: x[0])

In [33]:
x_train_LPC = []
x_valid_LPC = []
x_test_LPC = []

In [34]:
y_train = []
y_valid = []
y_test = []

In [35]:
x_train_MFCC = []
x_valid_MFCC = []
x_test_MFCC = []

In [36]:
for i in range(460):
    x_train_LPC.append(data_train_LPC[i][1])
for i in range(105):
    x_valid_LPC.append(data_valid_LPC[i][1])
for i in range(142):
    x_test_LPC.append(data_test_LPC[i][1])

In [37]:
for i in range(460):
    x_train_MFCC.append(data_train_MFCC[i][1])
for i in range(105):
    x_valid_MFCC.append(data_valid_MFCC[i][1])
for i in range(142):
    x_test_MFCC.append(data_test_MFCC[i][1])

In [38]:
for i in range(460):
    y_train.append(data_train_LPC[i][2])
for i in range(105):
    y_valid.append(data_valid_LPC[i][2])
for i in range(142):
    y_test.append(data_test_LPC[i][2])

In [39]:
y_train = [0 if label == 'Normal' else 1 for label in y_train]
y_test = [0 if label == 'Normal' else 1 for label in y_test]
y_valid = [0 if label == 'Normal' else 1 for label in y_valid]

In [40]:
Y_train = np.array(y_train)
Y_valid = np.array(y_valid)
Y_test = np.array(y_test)

In [41]:
Y_train

array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,

For having same frame numbers, here we add zero instead of features for samples that they have less frames than the 'Average + STD' of all samples' frame numbers.

In [42]:
z = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
Z = np.array(z)
for i in range (21):
  Z[i]=np.float64(Z[i])

In [43]:
x = [0,0,0,0,0,0,0,0,0,0,0,0,0]
X = np.array(x)
for i in range (13):
  X[i]=np.float64(X[i])

In [44]:
for i in range(460):
  for j in range(2040-len(x_train_LPC[i])):
     x_train_LPC[i] = np.append(x_train_LPC[i], [Z], axis=0)
for i in range(105):
  for j in range(2040-len(x_valid_LPC[i])):
     x_valid_LPC[i] = np.append(x_valid_LPC[i], [Z], axis=0)
for i in range(142):
  for j in range(2040-len(x_test_LPC[i])):
     x_test_LPC[i] = np.append(x_test_LPC[i], [Z], axis=0)

In [45]:
for i in range(460):
  for j in range(2045-len(x_train_MFCC[i])):
     x_train_MFCC[i] = np.append(x_train_MFCC[i], [X], axis=0)
for i in range(105):
  for j in range(2045-len(x_valid_MFCC[i])):
     x_valid_MFCC[i] = np.append(x_valid_MFCC[i], [X], axis=0)
for i in range(142):
  for j in range(2045-len(x_test_MFCC[i])):
     x_test_MFCC[i] = np.append(x_test_MFCC[i], [X], axis=0)

In [46]:
X_train_LPC = np.array(x_train_LPC)
X_valid_LPC = np.array(x_valid_LPC)
X_test_LPC = np.array(x_test_LPC)

In [47]:
X_train_MFCC = np.array(x_train_MFCC)
X_valid_MFCC = np.array(x_valid_MFCC)
X_test_MFCC = np.array(x_test_MFCC)

Normalization

In [48]:
import numpy as np

def z_score_normalize(data, mean=None, std=None):
    if mean is None:
        mean = np.mean(data, axis=(0, 1))
    if std is None:
        std = np.std(data, axis=(0, 1))
    normalized_data = (data - mean) / std
    return normalized_data, mean, std

# Assuming you have train_data, test_data, and valid_data


# Calculate mean and std from train_data and normalize all datasets
X_train_LPC, mean1, std1 = z_score_normalize(X_train_LPC)
X_test_LPC, _, _ = z_score_normalize(X_test_LPC, mean1, std1)
X_valid_LPC, _, _ = z_score_normalize(X_valid_LPC, mean1, std1)

X_train_MFCC, mean2, std2 = z_score_normalize(X_train_MFCC)
X_test_MFCC, _, _ = z_score_normalize(X_test_MFCC, mean2, std2)
X_valid_MFCC, _, _ = z_score_normalize(X_valid_MFCC, mean2, std2)

The Proposed CNN Model for Training our Datasets

In [67]:
model_LPC = Sequential()
# Convolutional Layers
model_LPC.add(Conv2D(16, (3, 3), input_shape=(2040, 20, 1), activation='relu', padding='same'))
#model.add(BatchNormalization())
model_LPC.add(MaxPooling2D(pool_size=(2, 2)))

model_LPC.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
#model.add(BatchNormalization())
model_LPC.add(MaxPooling2D(pool_size=(2, 2)))

model_LPC.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#model.add(BatchNormalization())
model_LPC.add(MaxPooling2D(pool_size=(2, 2)))

# Global Average Pooling
model_LPC.add(GlobalAveragePooling2D())

# Regularization
model_LPC.add(Dropout(0.5))

# Fully Connected Layers
model_LPC.add(Dense(128, activation='relu'))
model_LPC.add(Dropout(0.3))

model_LPC.add(Dense(1, activation='sigmoid'))

# Compile the model
model_LPC.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model_LPC.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 2040, 20, 16)        │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 1020, 10, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 1020, 10, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 510, 5, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 510, 5, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 255, 2, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,745 (124.00 KB)

 Trainable params: 31,745 (124.00 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model_MFCC = Sequential()
# Convolutional Layers
model_MFCC.add(Conv2D(16, (3, 3), input_shape=(2045, 13, 1), activation='relu', padding='same'))
#model.add(BatchNormalization())
model_MFCC.add(MaxPooling2D(pool_size=(2, 2)))

model_MFCC.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
#model.add(BatchNormalization())
model_MFCC.add(MaxPooling2D(pool_size=(2, 2)))

model_MFCC.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#model.add(BatchNormalization())
model_MFCC.add(MaxPooling2D(pool_size=(2, 2)))

# Global Average Pooling
model_MFCC.add(GlobalAveragePooling2D())

# Regularization
model_MFCC.add(Dropout(0.5))

# Fully Connected Layers
model_MFCC.add(Dense(128, activation='relu'))
model_MFCC.add(Dropout(0.3))

model_MFCC.add(Dense(1, activation='sigmoid'))

# Compile the model
model_MFCC.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model_MFCC.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 2045, 13, 16)        │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 1022, 6, 16)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 1022, 6, 32)         │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 511, 3, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 511, 3, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 255, 1, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,745 (124.00 KB)

 Trainable params: 31,745 (124.00 KB)

 Non-trainable params: 0 (0.00 B)

In [62]:
#saving the trained model for MFCC
filepath_MFCC = '/content/sample_data/model_test_MFCC.keras'
checkpoint = ModelCheckpoint(filepath=filepath_MFCC,
                             monitor='val_accuracy',
                             verbose=0,
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]

In [63]:
history = model_MFCC.fit(X_train_MFCC, Y_train, validation_data=(X_valid_MFCC, Y_valid),  batch_size=64, epochs=200, callbacks = callbacks)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 668ms/step - accuracy: 0.4767 - loss: 0.7040 - val_accuracy: 0.7619 - val_loss: 0.6716
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5696 - loss: 0.6724 - val_accuracy: 0.7524 - val_loss: 0.6458
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6326 - loss: 0.6580 - val_accuracy: 0.7810 - val_loss: 0.6117
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6710 - loss: 0.6297 - val_accuracy: 0.7905 - val_loss: 0.5847
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6500 - loss: 0.6364 - val_accuracy: 0.7333 - val_loss: 0.5666
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6988 - loss: 0.6024 - val_accuracy: 0.7810 - val_loss: 0.5430
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7328 - loss: 0.5894 - val_accuracy: 0.7714 - val_loss: 0.5343
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7068 - loss: 0.5700 - val_accuracy: 0.7905 - val_loss

In [64]:
from keras.models import load_model

model = load_model(filepath_MFCC)

Accuracies of the model for test and validation data by MFCC Features

In [65]:
test_loss, test_acc = model.evaluate(X_valid_MFCC,Y_valid, verbose=0)
print("validation Accuracy by MFCC: ",test_acc)

validation Accuracy by MFCC:  0.9047619104385376


In [66]:
test_loss, test_acc = model.evaluate(X_test_MFCC,Y_test, verbose=0)
print("Test Accuracy by MFCC: ",test_acc)

Test Accuracy by MFCC:  0.8591549396514893


In [68]:
#saving the trained model for lpc
filepath_LPC = '/content/sample_data/model_test_LPC.keras'
checkpoint = ModelCheckpoint(filepath=filepath_LPC,
                             monitor='val_accuracy',
                             verbose=0,
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]

In [69]:
history = model_LPC.fit(X_train_LPC, Y_train, validation_data=(X_valid_LPC, Y_valid),  batch_size=64, epochs=200, callbacks = callbacks)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 523ms/step - accuracy: 0.4875 - loss: 0.7042 - val_accuracy: 0.4762 - val_loss: 0.6805
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5187 - loss: 0.6803 - val_accuracy: 0.6762 - val_loss: 0.6753
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5454 - loss: 0.6791 - val_accuracy: 0.7333 - val_loss: 0.6696
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6038 - loss: 0.6797 - val_accuracy: 0.7048 - val_loss: 0.6617
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6172 - loss: 0.6770 - val_accuracy: 0.6762 - val_loss: 0.6570
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6320 - loss: 0.6707 - val_accuracy: 0.6762 - val_loss: 0.6407
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6315 - loss: 0.6603 - val_accuracy: 0.7143 - val_loss: 0.6223
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6230 - loss: 0.6675 - val_accuracy: 0.6857 - val_loss

In [70]:
from keras.models import load_model

model = load_model(filepath_LPC)

Accuracies of the model for test and validation data by LPC Features

In [71]:
test_loss, test_acc = model.evaluate(X_valid_LPC,Y_valid, verbose=0)
print("validation Accuracy by LPC: ",test_acc)

validation Accuracy by LPC:  0.8857142925262451


In [72]:
test_loss, test_acc = model.evaluate(X_test_LPC,Y_test, verbose=0)
print("test Accuracy by LPC: ",test_acc)

test Accuracy by LPC:  0.8169013857841492
